In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [3]:
# 加载数据集
data = pd.read_excel('data.xlsx')

# 提取特征和目标变量
X = data[['zALFF', 'zfALFF', 'zReHo']]
y = data['组']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# 定义参数范围
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'sigmoid','poly'],
    'gamma': [0.1, 1, 10]
}

# 创建SVC分类器
svc = SVC(random_state=42)

# 创建网格搜索对象
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5)

In [7]:
def calculate_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sen = tp / (tp + fn)
    spe = tn / (tn + fp)
    return acc, auc, sen, spe

In [11]:
# 初始化列表来保存每个epoch的性能指标和参数
epoch_range = range(1, 1001)
acc_list, auc_list, sen_list, spe_list = [], [], [], []
param_list = []

# 迭代训练每个epoch
for epoch in epoch_range:
    # 执行网格搜索和五折交叉验证
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_

    # 在测试集上进行预测
    y_pred = grid_search.predict(X_test)

    # 计算性能指标
    acc, auc, sen, spe = calculate_metrics(y_test, y_pred)

    # 记录性能指标和参数
    acc_list.append(acc)
    auc_list.append(auc)
    sen_list.append(sen)
    spe_list.append(spe)
    param_list.append(best_params)    
    
    print(f"Epoch {epoch + 1}: ACC={acc:.4f}, SEN={sen:.4f}, SPE={spe:.4f}, AUC={auc:.4f}")    

Epoch 2: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 3: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 4: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 5: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 6: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 7: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 8: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 9: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 10: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 11: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 12: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 13: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 14: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 15: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 16: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 17: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 18: ACC=0.8571, SEN=1.0000, SPE=0.8000, AUC=0.9000
Epoch 19: ACC=0.8571, SEN=1.0000, SPE=0

KeyboardInterrupt: 

In [6]:
# 绘制参数随epoch变化的曲线
plt.figure(figsize=(10, 6))
plt.plot(epoch_range, [params['C'] for params in param_list], label='C')
plt.plot(epoch_range, [params['gamma'] for params in param_list], label='gamma')
plt.xlabel('Epoch')
plt.ylabel('Parameter Value')
plt.title('Parameter Values over Epochs')
plt.legend()
plt.show()

# 绘制ROC曲线
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

LogisticRegression()